In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from pathlib import Path
import pandas as pd
import numpy as np
import os
from datetime import datetime
from os.path import isdir

In [3]:
from tddl.post_processing.path_utils import logdir_to_paths

logdir = Path("/bigdata/cifar10/logs/decomposed")
folders = os.listdir(logdir)
paths = logdir_to_paths(logdir)

print(len(paths))

In [20]:
import torch
from tddl.data.loaders import fetch_loaders
from tddl.trainer import Trainer
from tqdm import tqdm


def add_test_before_ft(paths, dataset = 'cifar10', data_dir = "/bigdata/cifar10", batch = 512, data_workers = 8,
    optimizer = None, writer = None, scheduler = None, save = None, tuning = None,):
    
    train_loader, valid_loader, test_loader = fetch_loaders(
        dataset=dataset,
        path=data_dir,
        batch_size=batch,
        data_workers=data_workers,
        valid_size=5000,
    )
    
    for path in tqdm(paths):
        if path.joinpath('test_before_training.json').is_file():
            with open(path.joinpath('results_before_training.json')) as f:
                results_before_training = json.load(f)

            with open(path.joinpath('results_approximation_error.json')) as f:
                results_approximation_error = json.load(f)

            with open(path.joinpath('test_before_training.json')) as f:
                test_before_training = json.load(f)

            # test_before_training = {"test_acc": 0.9137, "test_loss": 0.0013673128426074982}
            results_before_training.update(test_before_training)
            results_approximation_error['test_acc_before_ft'] = test_before_training['test_acc']

            # with open(path.joinpath('results_before_training.json'), 'w') as f:
            #     json.dump(results_before_training, f)
            with open(path.joinpath('results_approximation_error.json'), 'w') as f:
                json.dump(results_approximation_error, f)

            continue

        print(path)
        # load decomposed model 
        model_path = path / "model_after_fact.pth"

        model = torch.load(model_path)

        # load trainer
        trainer = Trainer(
            train_loader, valid_loader, test_loader,
            model, optimizer, writer, 
            scheduler=scheduler, save=save, tuning=tuning)

        # test 
        test_acc, test_loss = trainer.test(loader="test")

        # write results
        with open(path.joinpath('results_before_training.json')) as f:
            results_before_training = json.load(f)
        # with open(path.joinpath('results_before_training.json'), 'w') as f:
        #     results_before_training = json.load(f)

        test_before_training = {
                "test_acc": test_acc,
                "test_loss": test_loss,
            }

        results_before_training.update(test_before_training)

        with open(path.joinpath('results_before_training.json'), 'w') as f:
            json.dump(results_before_training, f)



        with open(path.joinpath('results_approximation_error.json')) as f:
            results_approximation_error = json.load(f)
        
        results_approximation_error['test_acc_before_ft'] = test_before_training['test_acc']

        with open(path.joinpath('results_approximation_error.json'), 'w') as f:
            json.dump(results_approximation_error, f)

        # with open(path.joinpath('test_before_training.json'), 'w') as f:
        #     json.dump(test_before_training, f)
        # append to "results_before_training.json"

In [19]:
from tddl.post_processing.path_utils import logdir_to_paths

logdir = Path("/bigdata/f_mnist/logs/garipov/decomposed")
folders = os.listdir(logdir)
paths = logdir_to_paths(logdir)

print(len(paths))

dataset = 'cifar10'
data_dir = "/bigdata/cifar10"

add_test_before_ft(paths, dataset, data_dir)

250
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/250 [00:00<?, ?it/s]

/bigdata/cifar10/logs/garipov/decomposed/1647854386/gar-lr-[2]-tucker-0.9-dTrue-iNone_bn_128_sgd_l0.0001_g0.0_sTrue


  0%|          | 1/250 [00:00<03:05,  1.34it/s]

/bigdata/cifar10/logs/garipov/decomposed/1647856653/gar-lr-[2]-tucker-0.9-dTrue-iNone_bn_128_sgd_l0.0001_g0.0_sTrue


...

 99%|█████████▉| 248/250 [02:58<00:01,  1.31it/s]

/bigdata/cifar10/logs/garipov/decomposed/1647859377/gar-lr-[4]-tucker-0.9-dTrue-iNone_bn_128_sgd_l0.0001_g0.0_sTrue


100%|█████████▉| 249/250 [02:59<00:00,  1.35it/s]

/bigdata/cifar10/logs/garipov/decomposed/1647777065/gar-lr-[10]-cp-0.25-dTrue-iNone_bn_128_sgd_l0.0001_g0.0_sTrue


100%|██████████| 250/250 [03:00<00:00,  1.39it/s]


In [21]:
logdir = Path("/bigdata/f_mnist/logs/garipov/decomposed")
folders = os.listdir(logdir)
paths = logdir_to_paths(logdir)

print(len(paths))

dataset = 'fmnist'
data_dir = "/bigdata/f_mnist"

add_test_before_ft(paths, dataset, data_dir)

250


  0%|          | 0/250 [00:00<?, ?it/s]

/bigdata/f_mnist/logs/garipov/decomposed/1647969903/gar-lr-[8]-cp-0.75-dTrue-iNone_bn_128_sgd_l0.0001_g0.0_sTrue


  0%|          | 1/250 [00:00<02:47,  1.49it/s]

/bigdata/f_mnist/logs/garipov/decomposed/1647966199/gar-lr-[10]-cp-0.75-dTrue-iNone_bn_128_sgd_l0.0001_g0.0_sTrue


...

 99%|█████████▉| 248/250 [02:33<00:01,  1.60it/s]

/bigdata/f_mnist/logs/garipov/decomposed/1647978009/gar-lr-[6]-cp-0.25-dTrue-iNone_bn_128_sgd_l0.0001_g0.0_sTrue


100%|█████████▉| 249/250 [02:33<00:00,  1.58it/s]

/bigdata/f_mnist/logs/garipov/decomposed/1647978964/gar-lr-[8]-cp-0.25-dTrue-iNone_bn_128_sgd_l0.0001_g0.0_sTrue


100%|██████████| 250/250 [02:34<00:00,  1.62it/s]
